# Lab | Web Scraping Single Page

In [1]:
import pandas as pd
import numpy as np
import requests
import re 

from bs4 import BeautifulSoup

In [2]:
url = "https://www.billboard.com/charts/hot-100/"

In [3]:
response = requests.get(url)

In [4]:
def get_data_parsed(response):
    if response.status_code != 200:
        return np.NaN
    
    return BeautifulSoup(response.content, "html.parser")

In [5]:
data = get_data_parsed(response)

In [6]:
#data

In [7]:
span_tags = data.find_all("span")

In [8]:
spans = []

for span in span_tags:
    if span.text.lstrip().rstrip() != '':
        spans.append(span.text)

In [9]:
new_spans = []

for span in spans:
    if "\n" in span:
        new_spans.append(span)

In [10]:
pattern_content = r'\n\t\n\t(.+?)\n'

In [11]:
pattern_headers = r'\n{2}\t(.+?)\n'

In [12]:
headers = []

In [13]:
for span in new_spans:
    if re.findall(pattern_headers, span):
        headers.append(span)

In [14]:
headers = headers[9:14]

In [15]:
headers = [header.replace('\n', '').replace('\t', '') for header in headers]

In [16]:
cols = ['Rank', 'Artist']

In [17]:
headers = headers[2:]

In [18]:
cols.extend(headers)

In [19]:
content = []

In [20]:
for span in new_spans:
    if re.findall(pattern_content, span):
        content.append(span)

In [21]:
content = content[3:]

In [22]:
content = [row.replace('\n', '').replace('\t', '') if 'NEW' not in row else '' for row in content ]

In [23]:
content = [row.replace('\n', '').replace('\t', '') if 'RE-ENTRY' not in row else '' for row in content ]

In [24]:
content = [row for row in content  if row != '']

In [25]:
subarrays = [content[i:i+8] for i in range(0, len(content), 8)]
    
songs = [song[0:5] for song in subarrays]

In [26]:
list_songs = pd.DataFrame(songs[:], columns=cols)

In [27]:
songs_soup = data.select("h3.a-no-trucate")

In [28]:
type(songs_soup)

bs4.element.ResultSet

In [29]:
final_songs = []

for song in songs_soup:
    final_songs.append(song.text.replace('\n', '').replace('\t', ''))

In [30]:
songs_df = pd.DataFrame(final_songs, columns=['Song'])

In [31]:
songs_df

,Song
0,Last Night
1,Fast Car
2,Meltdown
3,Cruel Summer
4,FE!N
...,...
95,Oh U Went
96,ICU
97,Pound Town 2
98,"Bzrp Music Sessions, Vol. 55"


In [32]:
final_data = pd.concat([list_songs, songs_df], axis=1)

In [33]:
final_data

,Rank,Artist,Last Week,Peak Pos.,Wks on Chart,Song
0,1,Morgan Wallen,2,1,27,Last Night
1,2,Luke Combs,3,2,19,Fast Car
2,3,Travis Scott Featuring Drake,-,3,1,Meltdown
3,4,Taylor Swift,6,4,13,Cruel Summer
4,5,Travis Scott Featuring Playboi Carti,-,5,1,FE!N
...,...,...,...,...,...,...
95,96,Young Thug Featuring Drake,66,19,6,Oh U Went
96,97,Coco Jones,71,62,18,ICU
97,98,Sexyy Red & Tay Keith & Nicki Minaj,74,66,9,Pound Town 2
98,99,Bizarrap & Peso Pluma,72,31,9,"Bzrp Music Sessions, Vol. 55"


In [34]:
cols = ['Rank', 'Song', 'Artist', 'Last Week', 'Peak Pos.', 'Wks on Chart']

In [35]:
final_data = final_data[cols]

In [36]:
final_data

,Rank,Song,Artist,Last Week,Peak Pos.,Wks on Chart
0,1,Last Night,Morgan Wallen,2,1,27
1,2,Fast Car,Luke Combs,3,2,19
2,3,Meltdown,Travis Scott Featuring Drake,-,3,1
3,4,Cruel Summer,Taylor Swift,6,4,13
4,5,FE!N,Travis Scott Featuring Playboi Carti,-,5,1
...,...,...,...,...,...,...
95,96,Oh U Went,Young Thug Featuring Drake,66,19,6
96,97,ICU,Coco Jones,71,62,18
97,98,Pound Town 2,Sexyy Red & Tay Keith & Nicki Minaj,74,66,9
98,99,"Bzrp Music Sessions, Vol. 55",Bizarrap & Peso Pluma,72,31,9


In [37]:
final_data.to_csv('./songs.csv')